## 1. Cree un RDD importes a partir de los datos adjuntos a esta lección como recurso. Emplee acumuladores para obtener el total de ventas realizadas y el importe total de las ventas.




In [10]:
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [11]:
importes = sc.textFile('files/rdd.txt')
importes.take(5)

['527', '386', '701', '240', '941']

In [15]:
total_importes = sc.accumulator(0)
cantidad_importes = sc.accumulator(0)

In [16]:
importes.foreach(lambda x: total_importes.add(float(x)))
importes.foreach(lambda x: cantidad_importes.add(1))
print('Total ventas = ',total_importes.value)
print('Cantidad de ventas = ',cantidad_importes.value)

Total ventas =  5042335.0
Cantidad de ventas =  10000


## 2. Si se conoce que a cada venta hay que restarle un importe fijo igual a 10 pesos por temas de impuestos.
### 2.1 ¿Cómo restaría este impuesto de cada venta utilizando una variable broadcast para acelerar el proceso?

In [18]:
impuesto = sc.broadcast(10)

In [19]:
ventas_con_impuestos = importes.map(lambda x: float(x) - impuesto.value)
print('ventas sin impuesto = ',importes.take(5))
print('ventas con impuesto = ',ventas_con_impuestos.take(5))

ventas sin impuesto =  ['527', '386', '701', '240', '941']
ventas con impuesto =  [517.0, 376.0, 691.0, 230.0, 931.0]


### 2.2 Destruya la variable broadcast creada luego de emplearla para crear el RDD del inciso b.

In [20]:
impuesto.destroy()

## 3. Persista el RDD ventas_sin_impuestos en los siguientes niveles de persistencia.
### 3.1 Memoria.

In [21]:
# Memoria actual
ventas_con_impuestos.cache()

PythonRDD[13] at RDD at PythonRDD.scala:53

por defecto siempre esta en MEMORY_ONLY

### 3.2 Disco solamente.

In [23]:
# desvincular nivel de memoria cache
ventas_con_impuestos.unpersist()

PythonRDD[13] at RDD at PythonRDD.scala:53

In [24]:
# Cambiar la nivel de la memoria cache
ventas_con_impuestos.persist(StorageLevel.DISK_ONLY)

PythonRDD[13] at RDD at PythonRDD.scala:53

### 3.3 Memoria y disco.

In [25]:
# desvincular nivel de memoria cache
ventas_con_impuestos.unpersist()

PythonRDD[13] at RDD at PythonRDD.scala:53

In [26]:
# Cambiar la nivel de la memoria cache
ventas_con_impuestos.persist(StorageLevel.MEMORY_AND_DISK_DESER)

PythonRDD[13] at RDD at PythonRDD.scala:53

In [9]:
sc.stop()